In [ ]:
from biogeodataframe import BioGeoDataFrame
from osgeo import gdal
import geopandas as gpd
from rioxarray.merge import merge_arrays
from geocube.api.core import make_geocube
import numpy as np
import math

In [ ]:
# Set the CRS to BC Albers
CRS = 'EPSG:3005'
GEOCUBE_RES = 500
N_SAMPLES = 5000

In [ ]:
# Read in species occurrence data as a geodataframe and remove non-georeferenced rows
species_tmp = gpd.read_file('../data/black_bear_occurrences.csv')
species_tmp = species_tmp[(species_tmp['decimalLatitude'] != '') & (species_tmp['decimalLongitude'] != '')]

In [ ]:
# Convert the geopandas to a BioGeoDataFrame, giving access to useful methods
N = np.nanmin((N_SAMPLES, species_tmp.shape[0]))
species_tmp = species_tmp.sample(N)

species = BioGeoDataFrame(species_tmp)
species = species.set_geometry(gpd.points_from_xy(
        species['decimalLongitude'], species['decimalLatitude'])).set_crs(4326)
species = species.to_crs(CRS)

In [ ]:
# Load in biogeoclimatic zones and reproject to desired CRS
# Use only the ZONE and geometry fields, the former of which is what we will predict species' distributions with
bec_tmp = gpd.read_file('../data/bec').to_crs(CRS)
bec_tmp = bec_tmp[['ZONE', 'SUBZONE', 'geometry']]

In [ ]:
# Categorical variables must be made numeric to be transformed into a raster, so must convert numbers back to strings
# To do this, create list of all strings
bec_zones = bec_tmp.ZONE.drop_duplicates().values.tolist()
bec_subzones = bec_tmp.SUBZONE.drop_duplicates().values.tolist()
categorical_enums = {"ZONE": bec_zones, "SUBZONE": bec_subzones}

In [ ]:
# Convert bec geodataframe to rioxarray raster
# Resolution is in the units of target CRS
bec = make_geocube(vector_data = bec_tmp, resolution=(GEOCUBE_RES, -GEOCUBE_RES), categorical_enums=categorical_enums)

In [ ]:
# print(np.unique(bec['ZONE']))
# print(np.unique(bec['ZONE'].astype(int)))

In [ ]:
# Convert numeric back to categorical string
######################################### DO NOT DELETE ######################################### 
# zone_string = bec['ZONE_categories'][bec['ZONE'].astype(int)].drop('ZONE_categories')
# bec['ZONE'] = zone_string

In [ ]:
# Create pseudo-absences
pres_abs = species.add_pseudo_absences(amount=species.shape[0], region_poly=bec_tmp)

In [ ]:
BUFFER_DISTANCE = bec.rio.resolution()[1] * 31.5 # in units of CRS

In [ ]:
# Given a list of raster tiles, find which ones intersect the species occurrence points and are therefore required
# Using a single raster, bec, for simplicity
rasters = pres_abs.list_rasters(BUFFER_DISTANCE, [bec])

In [ ]:
# Load the list of raster tiles into memory
# Would load the rasters here, but bec is already loaded for simplicity. Something like:
# rasters = [rioxarray.open_rasterio(x) for x in raster]
# merged_raster = merge_arrays(rasters)
merged_raster = bec

In [ ]:
# Buffer each point so it intersects adjacent raster cells
# pres_abs['buffered_geometry'] = pres_abs['geometry'].buffer(BUFFER_DISTANCE, cap_style=3)

In [ ]:
# For each occurrence point, build a 3D tensor
vals = pres_abs.extract_values(raster=merged_raster, distance=BUFFER_DISTANCE)
vals = np.concatenate(vals)

In [ ]:
# Import required packages
import tensorflow as tf
import keras
from keras import layers
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
# listt = []
# listt.append(np.array((((1, 1), (2, 2)), ((1, 1), (2, 2)))))
# arr2 = np.array((((2, 2), (4, 4)), ((4, 4), (4, 4))))

# # np.stack((listt, arr2)).shape
# np.stack((listt))
# # arr2

In [ ]:
# # [x['presence'] for x in vals if None not in x['arr'][0] and 'nodata' not in x['arr'][1]].__len__()
# l = [
#     x["arr"]
#     for x in vals
# ]
# vals
# # [x.shape for x in l]

In [ ]:
# np.array((((2, 2), (3, 3)), ((2, 2), (3, 3)), ((2, 2), (3, 3))))
# np.zeros((255,255,3))
# x_train[0].transpose().shape

In [ ]:
from matplotlib.pyplot import imshow

imshow([x["arr"].transpose() for x in vals][10][:,:,1])


In [ ]:
x_data = np.stack(
    [x["arr"].transpose() for x in vals]
)  # if None not in x['arr'] is not None and 'nodata' not in x['arr']], axis=0)
y_data = np.stack(
    [x["presence"] for x in vals]
)  # if x['arr'] is not None and 'nodata' not in x['arr']])

In [ ]:
x_train, x_test = (
    x_data[0 : math.ceil(x_data.shape[0] * 0.8)],
    x_data[math.ceil(x_data.shape[0] * 0.8) : -1],
)

y_train, y_test = (
    y_data[0 : math.ceil(x_data.shape[0] * 0.8)],
    y_data[math.ceil(x_data.shape[0] * 0.8) : -1],
)

In [33]:
# model = tf.keras.models.Sequential([
#   # tf.keras.layers.Input(shape=(1,)),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(4, activation='relu'),
#   tf.keras.layers.Dense(4, activation='relu'),
#   tf.keras.layers.Dense(2, activation='softmax')
# ])

model = tf.keras.models.Sequential()
model.add(Conv2D(32, (2, 2), input_shape=(64, 64, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # downsample each dimension by a factor of 2

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(2)) # This should be the number of layers
model.add(Activation('softmax'))
# len(model.weights)

In [34]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
m = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
125/125 [==============================] - 3s 20ms/step - loss: 12.0036 - accuracy: 0.5223 - val_loss: 1.7203 - val_accuracy: 0.5412
Epoch 2/10
125/125 [==============================] - 2s 18ms/step - loss: 6.5286 - accuracy: 0.5366 - val_loss: 0.8576 - val_accuracy: 0.6145
Epoch 3/10
125/125 [==============================] - 2s 18ms/step - loss: 4.4206 - accuracy: 0.5374 - val_loss: 1.2786 - val_accuracy: 0.5592
Epoch 4/10
125/125 [==============================] - 2s 18ms/step - loss: 3.1100 - accuracy: 0.5326 - val_loss: 1.7102 - val_accuracy: 0.5271
Epoch 5/10
125/125 [==============================] - 2s 18ms/step - loss: 2.2368 - accuracy: 0.5356 - val_loss: 1.4456 - val_accuracy: 0.5241
Epoch 6/10
125/125 [==============================] - 2s 18ms/step - loss: 1.7150 - accuracy: 0.5466 - val_loss: 1.7297 - val_accuracy: 0.5201
Epoch 7/10
125/125 [==============================] - 2s 18ms/step - loss: 1.3110 - accuracy: 0.5602 - val_loss: 1.0364 - val_accuracy: 0.547